# Wrangle and Analyze Data
# WeRateDogs Dataset

WeRateDogs is a twitter account that rates dogs. we are using data from their account for data wrangling course.<br>
A project which is part of Udacity Data Analyst Nanodegree.

#### Steps to Effective Data Wrangling are: 
1. We will first gather data from 3 differents sources using different methods
2. Then Assess the data visually and programatically for quality and tidiness issues. 
3. And then we are going to resolve those issues. 
4. The last step is no longer data wrangling, we are going to analyze the data. Give insights and make vizualisations. 


## Gathering data
We have 3 sources of data to gather for this analysis.
1. twitter-archive-enhanced.csv, this file was already provided as a csv file. we are going to use pandas read_csv to import the file
2. image_predictions.tsv by using request library to download it
3. By using tweepy, extract data  from twitter API and store it in a file named tweet_json.txt

In [1]:
# importing all the library that we will need
import pandas as pd
import numpy as np
import tweepy
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

### Importing file 1: twitter-archive-enhanced.csv

In [2]:
#import the twitter-archive-enhanced.csv
#Reading a sample to have a feel of the data
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
517,810896069567610880,NaN,NaN,2016-12-19 17:14:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Hunter. He just found out he needs bra...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/810896069...,11,10,Hunter,None,None,None,None
94,874012996292530176,NaN,NaN,2017-06-11 21:18:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sebastian. He can't see all the colors...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/874012996...,13,10,Sebastian,None,None,None,puppo
1200,716730379797970944,NaN,NaN,2016-04-03 20:53:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",There has clearly been a mistake. Pup did noth...,NaN,NaN,NaN,https://twitter.com/chpsanfrancisco/status/716...,12,10,None,None,None,None,None
2060,671182547775299584,NaN,NaN,2015-11-30 04:22:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This pup holds the secrets of the universe in ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671182547...,12,10,None,None,None,None,None
298,837012587749474308,NaN,NaN,2017-03-01 18:52:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @KennyFromDaBlok: 14/10 h*ckin good hats. w...,8.370113e+17,726634734.0,2017-03-01 18:47:10 +0000,https://twitter.com/KennyFromDaBlok/status/837...,14,10,None,None,None,None,None


### Using Request to programatically download  image_predictions.tsv from given url

In [3]:
#create a variable containing the url from which we want to request Data
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
#save the requested url in a file named image_predictions.tsv
with open('image_predictions.tsv', 'wb') as f:
    f.write(response.content)
#import the file in a dataframe with pandas
image_df = pd.read_csv('image_predictions.tsv', delimiter='\t')
image_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
348,672481316919734272,https://pbs.twimg.com/media/CVUiMUeW4AEQgkU.jpg,1,Border_collie,0.599454,True,collie,0.106227,True,Shetland_sheepdog,0.094465,True
379,673270968295534593,https://pbs.twimg.com/media/CVfwXuWWIAAqnoi.jpg,1,Shih-Tzu,0.610453,True,Maltese_dog,0.166815,True,Old_English_sheepdog,0.132015,True
962,705970349788291072,https://pbs.twimg.com/media/CcwcSS9WwAALE4f.jpg,1,golden_retriever,0.776346,True,Labrador_retriever,0.112413,True,chow,0.036953,True
339,672245253877968896,https://pbs.twimg.com/media/CVRLfeoW4AA_ldZ.jpg,1,Chihuahua,0.718944,True,badger,0.178546,False,toy_terrier,0.037103,True
1818,834167344700198914,https://pbs.twimg.com/media/C5OOxY6WAAAxERz.jpg,1,ox,0.991682,False,bison,0.005335,False,water_buffalo,0.001130,False


### Retrivieng Data from Twitter API

In [4]:
#uthentication with twitter API using 
auth = tweepy.OAuthHandler('p2BWpeoQLwgyO9HNBAeYAwSAu' ,'fRnHPR225sL9lQtgvANK3tZMVh3TrwkPw6TkRqHMsY3EWgHW7y')
auth.set_access_token('1090421968623390720-cenMGb4xHFMnuyBXBZuLH4QuCRIaKZ','XMGq0HEaSWrSXVQMT080gkPToTx6HwuH0X4AWCaVGlR4R')

#create api to create to twitter with credetianls
api = tweepy.API(auth , wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [5]:
#putting tweet ids from archive_df in a list. the list will be used to query twitter api for more data
tweet_ids = df_archive['tweet_id'].tolist()

In [6]:
#looping through the list of tweet_ids to query to twitter API, saving the changes to tweet.json file
with open('tweet_json.txt', 'a', encoding='utf8') as file:
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, file)
            file.write('\n')
        except tweepy.TweepError as e:
            #print (e.api_code)
            print (e.reason)

Rate limit reached. Sleeping for: 492


Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 600


Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))
[{'code': 144, 'message': 'No status found with that ID.'}]
[{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 628


Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))


In [15]:
#taking the api data from json file to panda dataframe
api_data = []

with open('tweet_json.txt') as json_file:
    for line in json_file:
        try: 
            json_data = json.loads(line)
            api_data.append(json_data)
        except:
            continue
api_data_df = pd.DataFrame(api_data)
df_archive.tweet_id.count() - api_data_df.id.count()

19

In [8]:
twitter_api_df = api_data_df[['id', 'retweet_count', 'favorite_count']]
twitter_api_df.head()

,id,retweet_count,favorite_count
0,892177421306343426,6124,32601
1,891815181378084864,4055,24551
2,891689557279858688,8425,41308
3,891327558926688256,9133,39500
4,891087950875897856,3037,19850


## Ressources and references: 

* Requests Official Documentation: http://docs.python-requests.org/en/master/user/quickstart/#response-content
* Dowloading files using Requests: https://www.geeksforgeeks.org/downloading-files-web-using-python/
* Importing tsv file: https://www.kaggle.com/tinoswe/split-python-dataframe
* Using Tweepy to query twitter API : http://docs.tweepy.org/en/3.7.0/getting_started.html